<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7014d2bc93eebe70a26b06d164cfca2c19a8f9452cc675cc5d31bee5526917fc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-02 10:42:17
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -50.12 K (-0.35%)
Current PnL: -23.52 L (-15.41%)
CY Booked + Current PnL: -9.63 L (-6.31%)
-------------------
Total profit:  1.46 L
Total loss:  -24.98 L
-------------------
Total Booked + Current PnL: 17.35 L (13.81%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.76%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.93 L (62.5%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.0%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.12,-8.52,9.33,0.02,14899.0,-14872.0,159687.0,143.30,67.0,M-SC,13.08,234.0,-1.00,1.12,38.90,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.70,-16.24,19.50,0.09,16460.0,-16367.0,84410.0,95.94,40.0,M-SC,3.01,253.0,-0.99,0.59,10.90,OX40N,NTT,DURABLES
51,MASFIN,398.61,0.86,-2.92,25.72,22.05,24464.0,-2865.0,95115.0,-16.44,57.0,H-SC,6.60,164.0,-0.12,0.67,38.26,XR,ATH,FINANCE
43,ITC,452.00,-0.68,-1.90,12.58,10.44,24699.0,-3804.0,196334.0,-41.56,41.0,X-LC,1.41,5.0,-0.15,1.38,3.56,X40,NTT,FMCG
35,ICICIGI,2252.93,-1.28,4.88,14.60,20.19,26693.0,8509.0,182829.0,-17.91,44.0,X-MC,6.90,68.0,0.32,1.29,20.73,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,2.89,-28.10,109.71,50.79,110953.0,-39520.0,101133.0,-7.61,61.0,H-SC,6.43,151.0,-0.36,0.71,17.75,XR,ATH,IT
84,WHIRLPOOL,2270.00,2.25,-18.85,124.18,81.92,115685.0,-21639.0,93159.0,-52.45,24.0,M-SC,3.92,236.0,-0.19,0.65,10.29,XR,NTT,DURABLES
10,BANDHANBNK,400.00,1.33,-26.86,163.07,92.42,332250.0,-74812.0,203747.0,43.68,44.0,H-SC,6.03,170.0,-0.23,1.43,17.75,XY24,NTT,BANKS
5,ASIANPAINT,3460.25,1.21,1.44,19.23,20.95,49112.0,3626.0,255394.0,0.35,76.0,X-LC,16.45,31.0,0.07,1.80,37.59,X40,ATH,PAINTS
67,SIS,528.00,1.13,-25.03,61.74,21.26,51395.0,-27787.0,83245.0,1962.69,44.0,H-SC,3.86,166.0,-0.54,0.59,12.47,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-6.80,-20.35,86.38,48.45,137951.0,-40803.0,159703.0,-26.10,13.0,X-MC,8.30,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
52,MEDANTA,1486.00,-3.15,0.75,23.16,24.08,29622.0,950.0,127900.0,-7.41,40.0,X-SC,5.94,89.0,0.03,0.90,20.51,XY24,NTT,HEALTHCARE
44,JCHAC,2282.00,-2.24,-30.64,44.17,-0.01,39153.0,-39163.0,88642.0,18030.43,12.0,M-SC,0.73,233.0,-1.00,0.62,0.37,OX40N,NTT,AC
38,INDIAMART,4810.62,-2.07,-1.92,110.76,106.72,133989.0,-2364.0,120972.0,-51.39,35.0,H-SC,1.97,139.0,-0.02,0.85,21.01,AR,ATH,MISC
81,VALIANTORG,838.00,-2.02,-46.49,225.44,74.15,197373.0,-76055.0,87550.0,-257.21,34.0,H-SC,18.87,149.0,-0.39,0.62,12.22,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,0.48,0.04,48.57,48.63,97157.0,81.0,200035.0,-33.40,64.0,H-SC,15.11,136.0,0.00,1.41,11.60,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,0.52,-0.74,36.64,35.63,76221.0,-1547.0,208027.0,-14.53,45.0,H-MC,3.66,119.0,-0.02,1.46,15.81,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.07,-4.89,118.61,107.91,166550.0,-7226.0,140418.0,-22.58,36.0,M-SC,10.95,216.0,-0.04,0.99,0.41,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.12,-8.52,9.33,0.02,14899.0,-14872.0,159687.0,143.30,67.0,M-SC,13.08,234.0,-1.00,1.12,38.90,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.70,-16.24,19.50,0.09,16460.0,-16367.0,84410.0,95.94,40.0,M-SC,3.01,253.0,-0.99,0.59,10.90,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.72,-23.09,47.54,13.47,98600.0,-62262.0,207405.0,-68.31,31.0,H-SC,1.97,158.0,-0.63,1.46,5.46,XY24,NTT,PAINTS
67,SIS,528.00,1.13,-25.03,61.74,21.26,51395.0,-27787.0,83245.0,1962.69,44.0,H-SC,3.86,166.0,-0.54,0.59,12.47,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.12,2.53,68.01,72.27,118163.0,4295.0,173743.0,-11.98,59.0,M-LC,3.55,99.0,0.04,1.22,8.87,XR,NTT,IT
37,IEX,219.0,0.48,0.04,48.57,48.63,97157.0,81.0,200035.0,-33.40,64.0,H-SC,15.11,136.0,0.00,1.41,11.60,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.12,2.53,68.01,72.27,118163.0,4295.0,173743.0,-11.98,59.0,M-LC,3.55,99.0,0.04,1.22,8.87,XR,NTT,IT
37,IEX,219.00,0.48,0.04,48.57,48.63,97157.0,81.0,200035.0,-33.40,64.0,H-SC,15.11,136.0,0.00,1.41,11.60,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,0.52,-0.74,36.64,35.63,76221.0,-1547.0,208027.0,-14.53,45.0,H-MC,3.66,119.0,-0.02,1.46,15.81,AR,ATH,PHARMA
38,INDIAMART,4810.62,-2.07,-1.92,110.76,106.72,133989.0,-2364.0,120972.0,-51.39,35.0,H-SC,1.97,139.0,-0.02,0.85,21.01,AR,ATH,MISC
25,FINCABLES,1641.55,0.07,-4.89,118.61,107.91,166550.0,-7226.0,140418.0,-22.58,36.0,M-SC,10.95,216.0,-0.04,0.99,0.41,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-6.80,-20.35,86.38,48.45,137951.0,-40803.0,159703.0,-26.10,13.0,X-MC,8.30,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.95,-37.76,117.25,35.20,93891.0,-48592.0,80078.0,4.98,18.0,X-SC,13.96,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,0.01,-8.61,37.94,26.07,56773.0,-14100.0,149640.0,-32.65,28.0,X-MC,10.26,55.0,-0.25,1.05,0.01,X40,ATH,APPARELS
8,AWL,485.00,-1.10,-20.09,92.08,53.50,222040.0,-60613.0,241138.0,-61.33,31.0,X-MC,2.59,58.0,-0.27,1.69,7.10,XY24,NTT,FMCG
55,PGHH,17907.65,-0.38,-4.85,40.57,33.76,77523.0,-9735.0,191085.0,-32.91,33.0,X-MC,4.26,57.0,-0.13,1.34,0.10,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.68,-1.90,12.58,10.44,24699.0,-3804.0,196334.0,-41.56,41.0,X-LC,1.41,5.0,-0.15,1.38,3.56,X40,NTT,FMCG
3,ACC,3906.0,-0.03,-22.25,111.14,64.15,205609.0,-52951.0,185000.0,-54.57,49.0,X-SC,1.53,82.0,-0.26,1.30,3.78,XY24,BTT,CEMENT
15,CAMS,4762.0,-1.09,1.30,23.19,24.79,59166.0,3280.0,255136.0,-3.47,47.0,X-SC,1.92,86.0,0.06,1.79,24.74,X40N,NTT,MISC
36,ICICIPRULI,790.0,-0.58,2.45,27.84,30.97,50407.0,4321.0,181059.0,-21.60,52.0,X-MC,2.16,75.0,0.09,1.27,15.31,X40,ATH,INSURANCE
66,SIEMENS,4671.5,0.99,-10.18,39.74,25.51,66427.0,-18940.0,167155.0,2.62,67.0,H-LC,2.37,49.0,-0.29,1.17,23.18,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.95,-37.76,117.25,35.20,93891.0,-48592.0,80078.0,4.98,18.0,X-SC,13.96,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-6.80,-20.35,86.38,48.45,137951.0,-40803.0,159703.0,-26.10,13.0,X-MC,8.30,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
56,QUESS,424.00,-1.86,-29.52,102.68,42.84,47043.0,-19191.0,45815.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
54,PAGEIND,51605.07,0.01,-8.61,37.94,26.07,56773.0,-14100.0,149640.0,-32.65,28.0,X-MC,10.26,55.0,-0.25,1.05,0.01,X40,ATH,APPARELS
55,PGHH,17907.65,-0.38,-4.85,40.57,33.76,77523.0,-9735.0,191085.0,-32.91,33.0,X-MC,4.26,57.0,-0.13,1.34,0.10,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.86,-29.52,102.68,42.84,47043.0,-19191.0,45815.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,0.25,-47.08,187.81,52.31,144270.0,-68343.0,76817.0,-43.66,37.0,X-SC,4.89,91.0,-0.47,0.54,2.82,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.95,-37.76,117.25,35.20,93891.0,-48592.0,80078.0,4.98,18.0,X-SC,13.96,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-3.15,0.75,23.16,24.08,29622.0,950.0,127900.0,-7.41,40.0,X-SC,5.94,89.0,0.03,0.90,20.51,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-0.24,-6.68,30.29,21.58,39613.0,-9360.0,130779.0,-52.96,35.0,X-MC,5.99,56.0,-0.24,0.92,15.78,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.14,-14.75,39.91,19.28,117712.0,-51012.0,294944.0,-25.57,58.0,X-LC,6.91,1.0,-0.43,2.07,9.04,X40,ATH,IT
41,INFY,2275.00,0.22,6.71,45.14,54.87,150713.0,20990.0,333878.0,-16.73,62.0,X-LC,2.49,2.0,0.14,2.35,15.55,X40,BTT,IT
76,TMPV,600.00,-0.74,-14.43,66.16,42.18,156482.0,-39889.0,236521.0,-23.08,34.0,X-LC,2.75,3.0,-0.25,1.66,2.45,XY24,NTT,AUTO
82,VBL,671.64,-0.38,-2.58,39.26,35.67,120806.0,-8135.0,307707.0,-14.21,62.0,X-LC,5.45,4.0,-0.07,2.16,10.92,X40N,ATH,FMCG
43,ITC,452.00,-0.68,-1.90,12.58,10.44,24699.0,-3804.0,196334.0,-41.56,41.0,X-LC,1.41,5.0,-0.15,1.38,3.56,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.44,-37.18,111.54,32.89,54098.0,-28706.0,48501.0,-699.18,61.0,L-MC,5.92,259.0,-0.53,0.34,33.57,XR,NTT,BANKS
6,ASIANTILES,137.00,0.14,-15.16,114.13,81.67,90837.0,-14219.0,79591.0,7155.56,44.0,L-SC,19.02,271.0,-0.16,0.56,54.13,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.59,-11.99,58.55,39.54,52128.0,-12128.0,89031.0,-31.16,65.0,M-SC,6.69,220.0,-0.23,0.63,25.54,AR,ATH,AUTO
51,MASFIN,398.61,0.86,-2.92,25.72,22.05,24464.0,-2865.0,95115.0,-16.44,57.0,H-SC,6.60,164.0,-0.12,0.67,38.26,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.93,-19.69,53.19,23.03,76498.0,-35251.0,143820.0,55.70,48.0,H-SC,9.83,167.0,-0.46,1.01,42.70,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.12,-8.52,9.33,0.02,14899.0,-14872.0,159687.0,143.30,67.0,M-SC,13.08,234.0,-1.00,1.12,38.90,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,1.21,1.44,19.23,20.95,49112.0,3626.0,255394.0,0.35,76.0,X-LC,16.45,31.0,0.07,1.80,37.59,X40,ATH,PAINTS
50,LTIM,7201.88,-0.50,11.92,17.64,31.66,48596.0,29339.0,275490.0,7.74,66.0,H-LC,14.43,43.0,0.60,1.94,52.86,X200,ATH,IT
13,BSOFT,831.70,2.89,-28.10,109.71,50.79,110953.0,-39520.0,101133.0,-7.61,61.0,H-SC,6.43,151.0,-0.36,0.71,17.75,XR,ATH,IT
40,INDUSINDBK,1800.00,0.44,-37.18,111.54,32.89,54098.0,-28706.0,48501.0,-699.18,61.0,L-MC,5.92,259.0,-0.53,0.34,33.57,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.99
1,25,44.89
2,50,75.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.41
MC    29.00
LC    25.59
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.70
X40      21.24
X40N     11.95
XR        9.64
XY25      9.12
AR        9.10
OX40N     7.59
X200      1.94
SR        0.97
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.96
X-MC    22.50
X-LC    20.27
M-SC    11.86
X-SC     8.20
H-MC     4.77
H-LC     3.11
L-SC     1.39
M-MC     1.39
M-LC     1.22
L-LC     0.99
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.82,-5.07,39.05
IT,13.94,-14.07,72.96
FINANCE,9.43,-15.71,64.60
MISC,7.41,-23.42,73.98
ELECTRICAL,6.03,-9.32,49.37
PAINTS,5.95,-9.07,25.28
INSURANCE,4.40,-1.13,36.78
PHARMA,3.89,-0.80,33.06
AUTO,3.39,-19.38,70.57


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3098196.0,21
XR,1290194.0,13
AR,1279355.0,10
X40,1015565.0,14
X40N,746304.0,9
OX40N,712326.0,10
XY25,343637.0,6
SR,281691.0,2
MH,76828.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3575680.0,25
M-SC,1372100.0,15
X-MC,1276930.0,16
X-LC,879263.0,11
X-SC,752050.0,8
H-MC,399553.0,3
L-SC,263116.0,3
M-LC,118163.0,1
H-LC,115023.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1227196.0      6
           AR         883601.0      5
M-SC       XY24       790233.0      6
H-SC       XR         776826.0      7
X-MC       X40        452652.0      7
X-LC       X40        387747.0      5
X-MC       XY24       380791.0      3
H-SC       OX40N      329538.0      4
M-SC       OX40N      295472.0      5
X-SC       X40N       294610.0      3
           XY24       282274.0      3
H-SC       SR         281691.0      2
X-MC       X40N       259372.0      4
X-LC       XY24       226388.0      2
H-MC       AR         208239.0      2
X-LC       X40N       192322.0      2
H-MC       XY24       191314.0      1
X-MC       XY25       184115.0      2
L-SC       XR         175800.0      2
X-SC       X40        175166.0      2
M-SC       XR         165307.0      2
           AR         121088.0      2
M-LC       XR         118163.0      1
L-SC       OX40N       87316.0      1
H-SC       MH          76828.0      1
X-LC       XY25        72806.0      2
H-LC       AR          66427.0      1
L-MC       XR          54098.0      1
H-LC       X200        48596.0      1
M-MC       XY25        47573.0      1
L-LC       XY25        39143.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
